# Load a converted MNIST dataset

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
from pathlib import Path
import random

import keras
from keras import layers

In [5]:
from sedpack.io import Dataset

## Load

In [6]:
dataset = Dataset(Path.home() / "Datasets/mnist_dataset")  # Load the dataset

### Custom transformations

Indicate what are the inputs and what are the outputs.
Convert the label to one-hot encoding.

In [9]:
def process_record(rec):
    output = rec["digit"]
    output = keras.layers.CategoryEncoding(
        num_tokens=10, output_mode="one_hot")(output)
    return rec["input"], output

## Load training and validation splits

In [10]:
# Load train and validation splits of the dataset
batch_size = 128
train_data = dataset.as_tfdataset(
    "train",
    batch_size=batch_size,
    process_record=process_record,
)
validation_data = dataset.as_tfdataset(
    "test",  # validation split
    batch_size=batch_size,
    process_record=process_record,
)

2024-03-05 16:37:19.829923: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 16:37:19.835002: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 16:37:19.835128: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## Train

### Get model

In [14]:
input_shape = (28, 28)
num_classes = 10

input_data = keras.Input(shape=input_shape, name="input")

x = input_data
x = layers.Reshape((*input_shape, 1))(x)
x = layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(num_classes, activation="softmax", name="digit")(x)

model = keras.Model(inputs=input_data, outputs=x)

model.summary()
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28)]          0         
                                                                 
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                        

### Train the model

In [16]:
steps_per_epoch = 100
epochs = 10
_ = model.fit(
    train_data,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_data,
    validation_steps=steps_per_epoch // 10,
)

# Evaluate the model on hodlout.
holdout_data = dataset.as_tfdataset(
    "holdout",
    batch_size=batch_size,
    process_record=process_record,
    repeat=False,  # Single iteration over the dataset.
)
score = model.evaluate(
    holdout_data,
    verbose=0,
)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {100 * score[1]:.2f}%")

Epoch 1/10


2024-03-05 16:40:11.451615: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-03-05 16:40:11.905654: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3eed172320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-05 16:40:11.905672: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-03-05 16:40:11.910693: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709656811.961349  681081 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


100/100 [==============================] - 2s 5ms/step - loss: 0.9058 - accuracy: 0.7254 - val_loss: 0.2447 - val_accuracy: 0.9344
Epoch 2/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2583 - accuracy: 0.9230 - val_loss: 0.1246 - val_accuracy: 0.9664
Epoch 3/10
100/100 [==============================] - 0s 2ms/step - loss: 0.1709 - accuracy: 0.9466 - val_loss: 0.1044 - val_accuracy: 0.9695
Epoch 4/10
100/100 [==============================] - 0s 2ms/step - loss: 0.1539 - accuracy: 0.9500 - val_loss: 0.0914 - val_accuracy: 0.9680
Epoch 5/10
100/100 [==============================] - 0s 2ms/step - loss: 0.1203 - accuracy: 0.9634 - val_loss: 0.0721 - val_accuracy: 0.9758
Epoch 6/10
100/100 [==============================] - 0s 2ms/step - loss: 0.1187 - accuracy: 0.9620 - val_loss: 0.0661 - val_accuracy: 0.9820
Epoch 7/10
100/100 [==============================] - 0s 2ms/step - loss: 0.1047 - accuracy: 0.9670 - val_loss: 0.0570 - val_accuracy: 0.9820
Epoch 8/10
100/10

2024-03-05 16:40:14.588169: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11696572845710480607
2024-03-05 16:40:14.588182: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18106730496546039685
2024-03-05 16:40:14.588188: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8288636617555051526
